In [2]:
using Printf, Random, JuMP, LinearAlgebra, Mosek, MosekTools

Random.seed!(2024);

## Initial Formulation

In [3]:
# General parameters
NUM_DRONES, NUM_ITEMS, NUM_TIMES = 1, 3, 5
NUM_WAREHOUSES, NUM_CLIENTS, NUM_CHARGING_STATIONS = 5, 5, 2
TIME_BUDGET = 100

ITEMS = Dict(i => Dict("weight" => rand(Float16)) for i in 1:NUM_ITEMS);

# Locations
WAREHOUSES = Dict(
    l => Dict("availability" => Dict(key => rand(UInt8) for key in keys(ITEMS)))
    for l in 1:NUM_WAREHOUSES
);
CLIENTS = Dict(
    l => Dict("order" => Dict(key => rand(UInt8) for key in keys(ITEMS))) 
    for l in NUM_WAREHOUSES+1:NUM_WAREHOUSES+NUM_CLIENTS
);
CHARGING_STATIONS = Dict(
    l => Dict("max_output" => 10*rand(Float16)) 
    for l in NUM_WAREHOUSES+NUM_CLIENTS+1:NUM_WAREHOUSES+NUM_CLIENTS+NUM_CHARGING_STATIONS
);
LOCATIONS = merge(WAREHOUSES, CLIENTS, CHARGING_STATIONS);

DRONES = Dict(
    d => Dict(
        "max_load" => 100,
        "init_battery" => 50*rand(Float16),
        "init_load" => Dict(i => 0 for i in keys(ITEMS)),
        "init_position" => BitArray([1; zeros(length(LOCATIONS)-1)]),
        "flying_speed" => 2,
        "charging_speed" => 5,
        "battery_usage" => rand(Float16),
        "battery_efficiency" => 0.99,
        "battery_leakage" => rand(Float16),
        "battery_capacity" => 100*rand(Float16),
        "rendezvous" => BitArray([1; zeros(length(LOCATIONS)-1)])
    )
    for d in 1:NUM_DRONES
);
TIMES = Vector{UInt8}(1:NUM_TIMES);

# Create distance matrix
DISTANCES = rand(Float16, (length(LOCATIONS), length(LOCATIONS)));
foreach(i -> DISTANCES[i,i] = 0, 1:length(LOCATIONS)) # null diagonal
DISTANCES = (DISTANCES .+ DISTANCES') ./ 2;

TIME_WEIGHTS = reverse(TIMES) ./ sum(TIMES);

In [4]:
# Scheduling problem
model = Model(Mosek.Optimizer);

# Variables
@variable(model, is_here[keys(DRONES), keys(LOCATIONS), keys(LOCATIONS), TIMES], Bin); # did drone moved between two location
@variable(model, picked[keys(DRONES), keys(WAREHOUSES) ,keys(ITEMS), TIMES] >= 0, Int); # amount of items picked from warehouse by drone
@variable(model, dropped[keys(DRONES), keys(CLIENTS), keys(ITEMS), TIMES] >= 0, Int); # amount of items deliverd at client by drone
@variable(model, charged[keys(DRONES), keys(CHARGING_STATIONS), TIMES] >= 0); # amount of energy recahrged at charging station by drone
@variable(model, is_ended[TIMES], Bin);

In [5]:
# States
@expression(# Warehouse availability for each item at each time 
    model, stock[l in keys(WAREHOUSES), i in keys(ITEMS), t in TIMES],
    WAREHOUSES[l]["availability"][i] - sum(picked[:,l,i,1:t])
);
@expression(# Clients orders for each item at each time 
    model, order[l in keys(CLIENTS), i in keys(ITEMS), t in TIMES],
    CLIENTS[l]["order"][i] - sum(dropped[:,l,i,1:t])
);
@expression(# Drones load for each item at each time 
    model, load[d in keys(DRONES), i in keys(ITEMS), t in TIMES],
    DRONES[d]["init_load"][i] + sum(picked[d,:,i,1:t]) - sum(dropped[d,:,i,1:t])
);
@expression(# Weight carried by drone
    model, weight[d in keys(DRONES), t in TIMES], 
    sum(load[d,i,t]*ITEMS[i]["weight"] for i in keys(ITEMS))
);
@expression(# Travelled distance
    model, travelled_distance[d in keys(DRONES), t in TIMES],
    sum(DISTANCES.*is_here[d,:,:,t])
);
@expression(# Recharged energy per drone
    model, energy_in[d in keys(DRONES), t in TIMES], sum(charged[d,:,t]) 
);
@expression(# Consumed energy
    model, energy_out[d in keys(DRONES), t in TIMES],
    DRONES[d]["battery_usage"]*(weight[d,t] + travelled_distance[d,t])
);
@expression(# Drones battery state (FINISH)
    model, battery[d in keys(DRONES), t in TIMES],
    DRONES[d]["battery_leakage"]^t * DRONES[d]["init_battery"] + DRONES[d]["battery_efficiency"]*sum(energy_in[d,1:t]) - (1/DRONES[d]["battery_efficiency"])*sum(energy_out[d,1:t])
);
@expression(# Elapsed time between time slots
    model, elapsed_time[d in keys(DRONES), t in TIMES],
    (1/DRONES[d]["flying_speed"])*travelled_distance[d,t] + (1/DRONES[d]["charging_speed"])*sum(charged[d,:,t])
);

In [10]:
# Location-action constraints
@constraint(# Pick no item if not at a warehouse
    model, [d in keys(DRONES), l in keys(WAREHOUSES), t in TIMES],
    sum(picked[d,l,:,t]) <= sum(WAREHOUSES[l]["availability"][i] for i in keys(ITEMS))*sum(is_here[d,:,l,t])
);
@constraint(# Deliver no item if not at a client
    model, [d in keys(DRONES), l in keys(CLIENTS), t in TIMES],
    sum(dropped[d,l,:,t]) <= sum(CLIENTS[l]["order"][i] for i in keys(ITEMS))*sum(is_here[d,:,l,t])
);
@constraint(# Charge no energy amount of not at charging station
    model, [d in keys(DRONES), l in keys(CHARGING_STATIONS), t in TIMES],
    charged[d,l,t] <= CHARGING_STATIONS[l]["max_output"]*sum(is_here[d,:,l,t])
);

# Constraints
## Expression nonnegativity
@constraint(model, stock .>= 0);
@constraint(model, order .>= 0);
@constraint(model, load .>= 0);
@constraint(model, battery .>= 0);

## General constraints
@constraint(# Drones cannot fly between two disconnected locations
    model, [d in keys(DRONES), l in keys(LOCATIONS), ll in keys(LOCATIONS), t in TIMES], 
    is_here[d,l,ll,t] <= (DISTANCES[l,ll] > 0)
);
@constraint(# Drones can arrive to one location only 
    model, [d in keys(DRONES), t in TIMES], sum(is_here[d,:,:,t]) == 1
);
@constraint(# Destination at t-1 == depature at t
    model, [d in keys(DRONES), l in keys(LOCATIONS), t in 2:TIMES[end]], 
    sum(is_here[d,:,l,t-1]) .== sum(is_here[d,l,:,t])
);
@constraint(# Maximum weight carried by drones
    model, [d in keys(DRONES), t in TIMES], 
    weight[d,t] <= DRONES[d]["max_load"]
);
@constraint(# Maximum output station
    model, [l in keys(CHARGING_STATIONS), t in TIMES],
    sum(charged[:,l,t]) <= CHARGING_STATIONS[l]["max_output"]
);
@constraint(# Max battery
    model, [d in keys(DRONES), t in TIMES], battery[d,t] <= DRONES[d]["battery_capacity"]
);

## Operational constraint
@constraint(# Time budget limit
    model, [d in keys(DRONES)], sum(elapsed_time[d,:]) <= TIME_BUDGET
);
@constraint(# The scheduling ends when no order is left (upper side)
    model, [t in TIMES], 
    sum(order[:,:,t]) <= 10e3 * (1 - is_ended[t])   # sum(CLIENTS[l]["order"][i] for l in keys(CLIENTS), i in keys(ITEMS)) 
);
@constraint(# The scheduling ends when no order is left (lower side)
    model, [t in TIMES], 
    sum(order[:,:,t]) >= is_ended[t] - 1
);
# @constraint(# Insist on is_ended: is_ended[t-1] -> is_ended[t]
#     model, [t in 2:TIMES[end]], 
#     is_ended[t] >= is_ended[t-1]
# );
@constraint(# Rendezvous after completion
    model, [d in keys(DRONES), l in keys(LOCATIONS), t in TIMES],
    sum(is_here[d,:,l,t]) - DRONES[d]["rendezvous"][l] <= 1 - is_ended[t]
);

# Optional? Drones do not stay on the same location for two consecutive time steps
@constraint(
    model, [d in keys(DRONES), l in keys(LOCATIONS), t in TIMES],
    is_here[d,l,l,t] == 0
);

In [11]:
@objective(
    model, Min, sum(TIME_WEIGHTS[t]*order[l,i,t] for l in keys(CLIENTS), t in TIMES, i in keys(ITEMS))
);

In [12]:
# Solve
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 1879            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 885             
  Matrix variables       : 0               
  Integer variables      : 875             

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.09, probing time =  0.06
Presolved problem: 820 variables, 306 constraints, 7807 non-zeros
Presolved problem: 145 general integer, 665 binary, 10 continuous
Clique table size: 64
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   1.0941400045e+03     NA          0.3   
0        1        1        0        1.33700000

## Refomulation

In [8]:
# Convexified and concavified matrices
DISTANCES_CVX = DISTANCES + (dropdims(sum(DISTANCES, dims=1), dims=1) |> diagm);
DISTANCES_CNC = DISTANCES - (dropdims(sum(DISTANCES, dims=1), dims=1) |> diagm);

In [9]:
# Scheduling problem
model = Model(Mosek.Optimizer);

# Variables
@variable(model, is_here[keys(DRONES), keys(LOCATIONS), TIMES], Bin); # did drone moved between two location
@variable(model, picked[keys(DRONES), keys(WAREHOUSES) ,keys(ITEMS), TIMES] >= 0, Int); # amount of items picked from warehouse by drone
@variable(model, dropped[keys(DRONES), keys(CLIENTS), keys(ITEMS), TIMES] >= 0, Int); # amount of items deliverd at client by drone
@variable(model, charged[keys(DRONES), keys(CHARGING_STATIONS), TIMES] >= 0); # amount of energy recahrged at charging station by drone
@variable(model, is_ended[TIMES], Bin);

In [10]:
# States
@expression(# Warehouse availability for each item at each time 
    model, stock[l in keys(WAREHOUSES), i in keys(ITEMS), t in TIMES],
    WAREHOUSES[l]["availability"][i] - sum(picked[:,l,i,1:t])
);
@expression(# Clients orders for each item at each time 
    model, order[l in keys(CLIENTS), i in keys(ITEMS), t in TIMES],
    CLIENTS[l]["order"][i] - sum(dropped[:,l,i,1:t])
);
@expression(# Drones load for each item at each time 
    model, load[d in keys(DRONES), i in keys(ITEMS), t in TIMES],
    DRONES[d]["init_load"][i] + sum(picked[d,:,i,1:t]) - sum(dropped[d,:,i,1:t])
);
@expression(# Weight carried by drone
    model, weight[d in keys(DRONES), t in TIMES], 
    sum(load[d,i,t]*ITEMS[i]["weight"] for i in keys(ITEMS))
);
@expression(# Travelled distance
    model, travelled_distance[d in keys(DRONES), t in TIMES],
    t == 1 ? DRONES[d]["init_position"]'*DISTANCES_CVX*is_here[d,:,t] : is_here[d,:,t-1]'*DISTANCES_CVX*is_here[d,:,t]
);
@expression(# Recharged energy per drone
    model, energy_in[d in keys(DRONES), t in TIMES], sum(charged[d,:,t]) 
);
@expression(# Consumed energy
    model, energy_out[d in keys(DRONES), t in TIMES],
    DRONES[d]["battery_usage"]*(weight[d,t] + travelled_distance[d,t])
);
@expression(# Drones battery state
    model, battery[d in keys(DRONES), t in TIMES],
    DRONES[d]["battery_leakage"]^t * DRONES[d]["init_battery"] + DRONES[d]["battery_efficiency"]*sum(energy_in[d,1:t]) - (1/DRONES[d]["battery_efficiency"])*sum(energy_out[d,1:t])
);
@expression(# Elapsed time between time slots
    model, elapsed_time[d in keys(DRONES), t in TIMES],
    (1/DRONES[d]["flying_speed"])*travelled_distance[d,t] + (1/DRONES[d]["charging_speed"])*sum(charged[d,:,t])
);

In [11]:
# Location-action constraints
@constraint(# Pick no item if not at a warehouse
    model, [d in keys(DRONES), i in keys(ITEMS), l in keys(WAREHOUSES), t in TIMES],
    sum(picked[d,l,:,t]) <= WAREHOUSES[l]["availability"][i]*is_here[d,l,t]
);
@constraint(# Deliver no item if not at a client
    model, [d in keys(DRONES), i in keys(ITEMS), l in keys(CLIENTS), t in TIMES],
    sum(dropped[d,l,:,t]) <= CLIENTS[l]["order"][i]*is_here[d,l,t]
);
@constraint(# Charge no energy amount of not at charging station
    model, [d in keys(DRONES), l in keys(CHARGING_STATIONS), t in TIMES],
    charged[d,l,t] <= CHARGING_STATIONS[l]["max_output"]*is_here[d,l,t]
);

# # Constraints
## Expression nonnegativity
@constraint(model, stock .>= 0);
@constraint(model, order .>= 0);
@constraint(model, load .>= 0);
@constraint(model, battery .>= 0);

# ## General constraints
# @constraint(# Drones cannot fly between two disconnected locations
#     model, [d in keys(DRONES), l in keys(LOCATIONS), ll in keys(LOCATIONS), t in TIMES], 
#     (t == 1 ? DRONES[d]["init_position"][l] : is_here[d,l,t-1]) + is_here[d,ll,t] <= (DISTANCES[l,ll] > 0)
# );
# @constraint(# Drones can arrive to one location only 
#     model, [d in keys(DRONES), t in TIMES], sum(is_here[d,:,t]) == 1
# );
# @constraint(# Maximum weight carried by drones
#     model, [d in keys(DRONES), t in TIMES], 
#     weight[d,t] <= DRONES[d]["max_load"]
# );
# @constraint(# Maximum output station
#     model, [l in keys(CHARGING_STATIONS), t in TIMES],
#     sum(charged[:,l,t]) <= CHARGING_STATIONS[l]["max_output"]
# );
# @constraint(# Max battery
#     model, [d in keys(DRONES), t in TIMES], battery[d,t] <= DRONES[d]["battery_capacity"]
# );

## Operational constraint
# @constraint(# Time budget limit
#     model, [d in keys(DRONES)], sum(elapsed_time[d,:]) <= TIME_BUDGET
# );
# @constraint(# The scheduling ends when no order is left (upper side)
#     model, [t in TIMES], 
#     sum(order[:,:,t]) <= 10e3 * (1 - is_ended[t])   # sum(CLIENTS[l]["order"][i] for l in keys(CLIENTS), i in keys(ITEMS)) 
# );
# @constraint(# The scheduling ends when no order is left (lower side)
#     model, [t in TIMES], 
#     sum(order[:,:,t]) >= is_ended[t] - 1
# );
# @constraint(# Insist on is_ended: is_ended[t-1] -> is_ended[t]
#     model, [t in 2:TIMES[end]], 
#     is_ended[t] >= is_ended[t-1]
# );
# @constraint(# Rendezvous after completion
#     model, [d in keys(DRONES), l in keys(LOCATIONS), t in TIMES],
#     is_here[d,l,t] - DRONES[d]["rendezvous"][l] <= 1 - is_ended[t]
# );

# # Optional? Drones do not stay on the same location for two consecutive time steps
# @constraint(
#     model, [d in keys(DRONES), l in keys(LOCATIONS), t in 2:TIMES[end]],
#     is_here[d,l,t] + is_here[d,l,t-1] <= 1
# );

In [12]:
@objective(
    model, Min, sum(TIME_WEIGHTS[t]*order[l,i,t] for l in keys(CLIENTS), t in TIMES, i in keys(ITEMS))
);

In [13]:
# Solve
optimize!(model)

LoadError: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.LessThan{Float64}` constraint is not supported by the model: Unable to transform a quadratic constraint into a second-order cone constraint because the quadratic constraint is not strongly convex.

Convex constraints that are not strongly convex (that is, the matrix is positive semidefinite but not positive definite) are not supported yet.

Note that a quadratic equality constraint is non-convex.